In [ ]:
import os

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jul 28 14:31:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/NGCF-PyTorch/NGCF/')

In [ ]:
!python main.py --dataset amazon-book --regs [1e-5] --embed_size 64 --layer_size [64,64,64] --lr 0.0005 --save_flag 1 --pretrain 0 --batch_size 1024 --epoch 200 --verbose 50 --node_dropout [0.1] --mess_dropout [0.1,0.1,0.1]

n_users=52643, n_items=91599
n_interactions=2984108
n_train=2380730, n_test=603378, sparsity=0.00062
already load adj matrix (144242, 144242) 0.6714670658111572
Epoch 0 [487.7s]: train==[618.34045=618.23077 + 0.10878]
Epoch 9 [488.6s + 1721.9s]: train==[166.93042=166.71812 + 0.21227], recall=[0.02439, 0.08381], precision=[0.01072, 0.00771], hit=[0.16965, 0.43255], ndcg=[0.01889, 0.03940]
save the weights in path:  model/9.pkl
Epoch 19 [492.2s + 1726.5s]: train==[97.12160=96.82020 + 0.30150], recall=[0.02563, 0.08794], precision=[0.01114, 0.00798], hit=[0.17498, 0.43983], ndcg=[0.01969, 0.04114]
save the weights in path:  model/19.pkl
Epoch 29 [492.7s + 1719.2s]: train==[70.42224=70.05193 + 0.37031], recall=[0.02724, 0.09279], precision=[0.01174, 0.00832], hit=[0.18256, 0.45366], ndcg=[0.02110, 0.04364]
save the weights in path:  model/29.pkl
Epoch 39 [493.9s + 1931.1s]: train==[55.26741=54.83919 + 0.42818], recall=[0.02865, 0.09600], precision=[0.01224, 0.00860], hit=[0.18764, 0.46443]

In [1]:
path = '../Data/amazon-book/'
train_file = path + '/train.txt'
test_file = path + '/test.txt'

#get number of users and items
n_users, n_items = 0, 0
n_train, n_test = 0, 0
neg_pools = {}

exist_users = []

In [2]:
with open(train_file) as f:
    for l in f.readlines():
        if len(l) > 0:
            l = l.strip('\n').split(' ')
            items = [int(i) for i in l[1:]]
            uid = int(l[0])
            exist_users.append(uid)
            n_items = max(n_items, max(items))
            n_users = max(n_users, uid)
            n_train += len(items)

ValueError: invalid literal for int() with base 10: ''

In [3]:
with open(test_file) as f:
    for l in f.readlines():
        if len(l) > 0:
            l = l.strip('\n')
            try:
                items = [int(i) for i in l.split(' ')[1:]]
            except Exception:
                continue
            n_items = max(n_items, max(items))
            n_test += len(items)

In [4]:
n_items += 1
n_users += 1

def print_statistics():
    print('n_users=%d, n_items=%d' % (n_users, n_items))
    print('n_interactions=%d' % (n_train + n_test))
    print('n_train=%d, n_test=%d, sparsity=%.5f' % (n_train, n_test, (n_train + n_test)/(n_users * n_items)))

In [5]:
print_statistics()

n_users=13647, n_items=91599
n_interactions=801466
n_train=198088, n_test=603378, sparsity=0.00064


In [12]:
import scipy.sparse as sp
import numpy as np
R = sp.dok_matrix((n_users, n_items), dtype=np.float32)

In [16]:
train_items, test_set = {}, {}
with open(train_file) as f_train:
    with open(test_file) as f_test:
        for l in f_train.readlines():
            if len(l) == 0:
                break
            l = l.strip('\n')
            items = [int(i) for i in l.split(' ')]
            uid, train_items = items[0], items[1:]

            for i in train_items:
                R[uid, i] = 1.
                # R[uid][i] = 1
            try:
                train_items[uid] = train_items
            except Exception:
                continue

        for l in f_test.readlines():
            if len(l) == 0: break
            l = l.strip('\n')
            try:
                items = [int(i) for i in l.split(' ')]
            except Exception:
                continue

            uid, test_items = items[0], items[1:]
            test_set[uid] = test_items

In [19]:
from time import time
def get_adj_mat():
    try:
        t1 = time()
        adj_mat = sp.load_npz(path + '/s_adj_mat.npz')
        norm_adj_mat = sp.load_npz(path + '/s_norm_adj_mat.npz')
        mean_adj_mat = sp.load_npz(path + '/s_mean_adj_mat.npz')
        print('already load adj matrix', adj_mat.shape, time() - t1)

    except Exception:
        adj_mat, norm_adj_mat, mean_adj_mat = create_adj_mat()
        sp.save_npz(path + '/s_adj_mat.npz', adj_mat)
        sp.save_npz(path + '/s_norm_adj_mat.npz', norm_adj_mat)
        sp.save_npz(path + '/s_mean_adj_mat.npz', mean_adj_mat)
    return adj_mat, norm_adj_mat, mean_adj_mat

In [20]:
plain_adj, norm_adj, mean_adj = get_adj_mat()

already load adj matrix (144242, 144242) 0.48089098930358887


In [24]:
print(mean_adj.shape)

(144242, 144242)


In [33]:
import random as rd
def sample_pos_items_for_u(u, num):
    # sample num pos items for u-th user
    pos_items = train_items[u]
    n_pos_items = len(pos_items)
    pos_batch = []
    while True:
        if len(pos_batch) == num:
            break
        pos_id = np.random.randint(low=0, high=n_pos_items, size=1)[0]
        pos_i_id = pos_items[pos_id]

        if pos_i_id not in pos_batch:
            pos_batch.append(pos_i_id)
    return pos_batch

In [35]:
def sample_neg_items_for_u(u, num):
    # sample num neg items for u-th user
    neg_items = []
    while True:
        if len(neg_items) == num:
            break
        neg_id = np.random.randint(low=0, high=n_items,size=1)[0]
        if neg_id not in train_items[u] and neg_id not in neg_items:
            neg_items.append(neg_id)
    return neg_items

In [36]:
def sample_neg_items_for_u_from_pools(u, num):
    neg_items = list(set(neg_pools[u]) - set(train_items[u]))
    return rd.sample(neg_items, num)

In [39]:
def sample():
    if batch_size <= n_users:
        users = rd.sample(exist_users, batch_size)
    else:
        users = [rd.choice(exist_users) for _ in range(batch_size)]
    #print(users)
    pos_items, neg_items = [], []
    for u in users:
        pos_items += sample_pos_items_for_u(u, 1)
        neg_items += sample_neg_items_for_u(u, 1)

    return users, pos_items, neg_items

In [44]:
print(train_items)
#sample_pos_items_for_u(15817, 1)

[37931, 24141, 31310, 32608, 32776, 1923, 7140, 8850, 551, 1352, 7156, 1006, 4089, 9163, 12653, 6315, 1868, 7230, 975, 1435, 5406, 23186, 10690, 10874]


In [40]:
batch_size = 1024
print(n_users)
users, pos_items, neg_items = sample()

52644
[15817, 42849, 31781, 566, 21495, 19814, 41041, 9052, 38063, 35797, 40103, 1031, 29923, 42542, 6376, 39761, 37366, 6796, 6024, 22377, 29151, 38151, 5128, 36302, 22079, 39361, 34908, 30657, 24127, 9758, 48542, 25239, 35126, 5831, 25531, 4334, 7254, 42859, 26612, 11365, 5913, 9813, 32748, 45742, 2980, 11015, 23021, 31522, 1610, 44559, 25040, 17293, 28714, 21657, 14930, 21970, 46743, 37327, 52506, 18040, 27243, 7962, 27445, 40749, 15210, 20790, 48230, 11314, 19011, 4184, 19873, 44805, 20831, 29205, 21405, 7848, 29631, 30354, 1339, 6942, 18195, 50013, 34976, 37639, 44678, 35153, 7210, 5815, 31470, 17018, 12392, 8156, 26128, 6896, 7312, 9050, 14927, 29668, 41247, 34764, 29303, 39641, 10489, 45283, 3534, 50400, 37208, 44527, 26400, 14851, 17720, 42566, 24056, 2232, 48462, 33796, 48318, 17292, 13949, 35656, 5711, 22721, 45633, 25840, 21704, 10519, 34264, 2559, 12794, 40979, 47471, 41700, 47319, 41631, 13478, 30150, 33609, 11514, 18709, 2152, 38405, 3580, 32109, 19778, 32009, 24788, 3121

IndexError: list index out of range

In [51]:
print(n_users+n_items)

144244


In [57]:
import numpy as np
from scipy.sparse import dok_matrix
S = dok_matrix((5, 5), dtype=np.float32)

for i in range(5):
    for j in range(5):
        S[i, j] = i + j
        
print(S.todense())

[[0. 1. 2. 3. 4.]
 [1. 2. 3. 4. 5.]
 [2. 3. 4. 5. 6.]
 [3. 4. 5. 6. 7.]
 [4. 5. 6. 7. 8.]]


In [85]:
import scipy.sparse as sp
import torch
adj_mat = sp.load_npz(path + 's_adj_mat.npz')
rowsum = np.array(adj_mat.sum(1))
print(adj_mat)

rowsum = np.array(adj_mat.sum(1))
d_inv = np.power(rowsum, -0.5).flatten()
d_inv[np.isinf(d_inv)] = 0.
d_mat_inv = sp.diags(d_inv)

norm_adj = d_mat_inv.dot(adj_mat)
norm_adj = norm_adj.dot(d_mat_inv)
print('generate pre adjacency matrix.')
pre_adj_mat = norm_adj.tocsr()

  (0, 52643)	1.0
  (0, 52644)	1.0
  (0, 52645)	1.0
  (0, 52646)	1.0
  (0, 52647)	1.0
  (0, 52648)	1.0
  (0, 52649)	1.0
  (0, 52650)	1.0
  (0, 52651)	1.0
  (0, 52652)	1.0
  (0, 52653)	1.0
  (0, 52654)	1.0
  (0, 52655)	1.0
  (0, 52656)	1.0
  (0, 52657)	1.0
  (0, 52658)	1.0
  (0, 52659)	1.0
  (0, 52660)	1.0
  (0, 52661)	1.0
  (0, 52662)	1.0
  (0, 52663)	1.0
  (0, 52664)	1.0
  (0, 52665)	1.0
  (0, 52666)	1.0
  (0, 52667)	1.0
  :	:
  (144218, 52477)	1.0
  (144218, 52493)	1.0
  (144219, 52477)	1.0
  (144220, 52477)	1.0
  (144221, 52477)	1.0
  (144222, 52477)	1.0
  (144223, 52477)	1.0
  (144224, 52477)	1.0
  (144225, 52477)	1.0
  (144226, 52477)	1.0
  (144227, 52477)	1.0
  (144228, 52481)	1.0
  (144229, 52481)	1.0
  (144230, 52487)	1.0
  (144231, 52489)	1.0
  (144232, 52495)	1.0
  (144233, 52502)	1.0
  (144234, 52502)	1.0
  (144235, 52503)	1.0
  (144236, 52504)	1.0
  (144237, 52506)	1.0
  (144238, 52506)	1.0
  (144239, 52506)	1.0
  (144240, 52508)	1.0
  (144241, 52508)	1.0
